In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold


In [20]:
# Read in the MNIST sample
data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/DataSets/mnistTrain.csv")

# Randomly choose a row, ignore first column (the label), transform into an array
label = data.iloc[6,0]
image = data.iloc[6,1:].values.reshape(28,28)
px.imshow(image, color_continuous_scale ='Greys')

Error in callback <bound method PositronShell._handle_post_run_cell of <positron.positron_ipkernel.PositronShell object at 0x000001E1500B81D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1e177595610, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1e17757a7e0, raw_cell="# Read in the MNIST sample
data = pd.read_csv("htt.." store_history=True silent=False shell_futures=True cell_id=None> result=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f8',
                    'shape': '28, 28'}}],
    'layout': {'coloraxis': {'colorscale': [[0.0, 'rgb(255,255,255)'], [0.125,
                            

KeyboardInterrupt: 

In [ ]:
## Random Tree
# Load data, then separate x and y variables
Y = data['Label']
X = data.drop('Label', axis = 1)
# Randomly sample our data --> 90% to train with, and 10% for testing
x, xt, y, yt = train_test_split(X, Y, test_size=0.1, random_state = 42)
# "random_state" : Seed for the random number generator. If you remove it, you’ll get a different split each run. 42 is just a convention — any integer works.
# fixating seed is important to see "my model is improving because I used different models, not because I got a good random draw"

# Simple Decision Tree
tree = DecisionTreeClassifier(max_depth =5, min_samples_leaf=10)
tclf = tree.fit(x, y)
tpred = tclf.predict(xt)
print("\nThe decision tree has an accuracy of : %s\n"% str(accuracy_score(tpred, yt)))
# Here the baseline accuracy would be 10% choosing right numbers from 0-9

# Bagging Ensemble
bag = BaggingClassifier(n_estimators=100, n_jobs=-1, random_state=42) 
# "n_estimators" = Number of decision trees in the forest.---> More trees → lower variance, more stable predictions, ↑ training time, 100 is enough
# "n_jobs" = How many CPU cores to use during training and prediction. “How many workers build trees in parallel?" Feeding all at once? or one-or-two...
# n_jobs = -1 → use all available cores
# n_jobs = 1 → single-core (default in some environments)
baclf = bag.fit(x, y)
bapred = baclf.predict(xt)
print("\nThe bagging algorithm has an accuracy of : %s\n"% str(accuracy_score(bapred, yt)))

# Random Forest
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
fclf = forest.fit(x,y)
fpred = fclf.predict(xt)
print("\nThe random forest has an accuracy of : %s\n"% str(accuracy_score(fpred, yt)))



The decision tree has an accuracy of : 0.65


The bagging algorithm has an accuracy of : 0.906


The random forest has an accuracy of : 0.934



In [ ]:

# Excercise
train = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/DataSets/mnistTrain.csv")
test = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/DataSets/mnistTest.csv")

trY = train['Label']
trX = train.drop('Label', axis = 1)
teY = test['Label']
teX = test.drop('Label', axis = 1)

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
Eclf = forest.fit(trX, trY)
Epred = Eclf.predict(teX)
print("\nThe 100 random forest has an accuracy of : %s\n"% str(accuracy_score(Epred, teY)))

forest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42)
Eclf = forest.fit(trX, trY)
Epred = Eclf.predict(teX)
print("\nThe 1000 new random forest has an accuracy of : %s\n"% str(accuracy_score(Epred, teY)))

forest = RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=42)
Eclf = forest.fit(trX, trY)
Epred = Eclf.predict(teX)
print("\nThe 10000 new random forest has an accuracy of : %s\n"% str(accuracy_score(Epred, teY)))


The 100 random forest has an accuracy of : 0.9398


The 1000 new random forest has an accuracy of : 0.9426


The 10000 new random forest has an accuracy of : 0.9422



In [17]:
## Extreme Boosting; Boosted Trees
Y = data['Label']
X = data.drop('Label', axis = 1)
x, xt, y, yt = train_test_split(X, Y, test_size=0.1, random_state = 42) 

xgb = XGBClassifier(
    n_estimators=50, # The number of boosting rounds (i.e., the number of trees built sequentially). how many times you're going to iterate (revise) the model 
    max_depth=3, 
    learning_rate=0.5, # Also called eta. It shrinks how much each tree contributes; controling a learning speed; whether learning is aggressive (refining a lot) or not. 
    objective='multi:softmax' #Provides outputs directly rather than probabilities.'multi:softprob' will return probabilities. 
)

# Fit to our training split
xgb.fit(x, y)

# Make predictions based on the testing x values
pred = xgb.predict(xt)

# Print out the accuracy score
print(f"Accuracy score: {accuracy_score(yt, pred)*100:.2f}%")

Accuracy score: 93.00%


In [ ]:
## Cross-Validation

Y = data['Label']
X = data.drop('Label', axis = 1)
x, xt, y, yt = train_test_split(X, Y, test_size=0.1, random_state = 42) 
kf = KFold(n_splits=10) # 10 "Folds"

models = [] # We will store our models here

for train, test in kf.split(x): # Iterate over folds
    model = XGBClassifier(n_estimators=50, max_depth=3, learning_rate=0.5, objective='multi:softmax').fit(x.values[train], y.values[train]) # Fit model
    accuracy = accuracy_score(y.values[test],    # Store accuracy
        model.predict(x.values[test]))
    print("Accuracy: ", accuracy)            # Print results
    models.append([model, accuracy])      # Store it all

print("Mean Model Accuracy: ", np.mean([model[1] for model in models]))
print("Model Accuracy Standar Deviation: ", np.std([model[1] for model in models]))


Accuracy:  0.9288888888888889
Accuracy:  0.9355555555555556
Accuracy:  0.9266666666666666
Accuracy:  0.9266666666666666
Accuracy:  0.8955555555555555
Accuracy:  0.9377777777777778
Accuracy:  0.9266666666666666
Accuracy:  0.9444444444444444
Accuracy:  0.9288888888888889
Accuracy:  0.9355555555555556
Mean Model Accuracy:  0.9286666666666668
Model Accuracy Standar Deviation:  0.012390757208400582


# Assignment 2
## Econ 8310 - Business Forecasting

This assignment will make use of the models covered in Lessons 4 to 6. Models include:

- Decision Trees
- Random Forests
- Boosted Trees

Your job will be to forecast whether or not an individual purchased a “meal” at Stedman’s Café. You may need to restructure the data in order to make appropriate forecasts, but I have done significant preparation of the data to streamline the cleaning process. You will be graded based on the following:

- Your code executing without errors
- Storing your models to make predictions on new data
- Making reasonable predictions based on the data provided
- Forecasting whether or not each new observation is a meal

To complete this assignment, your code will need to contain the following:
1. A forecasting algorithm named `model` using the sklearn or xgboost implementation of one of the models covered in lessons 4 to 6. Your dependent variable is the `meal` variable (indicating whether or not a purchase was classified as a meal or not), and may or may not use exogenous variables from the remainder of the dataset. The training data is stored at the following URL: https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3.csv
2. A fitted model named `modelFit`. This should be a model instance capable of generating forecasts using new data in the same shape as the data used in part (1).
3. A list or series of predictions using the data from the test period named `pred`. Your data should simply contain binary values (`1` or `0`, not boolean values) for whether or not you expect that an individual purchased a meal.
Make sure that your forecast includes predictions for EACH observation! There are 1000 observations in the prediction timeframe. The data from which to generate predictions is stored at the following URL: https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3test.csv

Note: While all models from lessons 4 to 6 are available to you, they may not all be good fits for the data. I recommend considering the data carefully, then choosing models to try. See which models seem to perform best on this data, and implement the best performer for the final submission of the project.

All code for this assignment should be written in the `assignment2.py` file found in the filetree.


In [5]:
train = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/AssignmentData/assignment3.csv")
test = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/AssignmentData/assignment3test.csv")

# Just Exploring...
px.line(train, x='DateTime', y='meal')


In [11]:
train = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/AssignmentData/assignment3.csv")
test = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/refs/heads/master/AssignmentData/assignment3test.csv")

train['DateTime'] = pd.to_datetime(train['DateTime'])
train['year'] = train['DateTime'].dt.year
train['month'] = train['DateTime'].dt.month
train['day'] = train['DateTime'].dt.weekday
train['hour'] = train['DateTime'].dt.hour
train.head()

test['DateTime'] = pd.to_datetime(test['DateTime'])
test['year'] = test['DateTime'].dt.year
test['month'] = test['DateTime'].dt.month
test['day'] = test['DateTime'].dt.weekday
test['hour'] = test['DateTime'].dt.hour
test.head()

,id,DateTime,Total,Discounts,Brewed_Coffee_12_oz,Brewed_Coffee_16_oz,Aquafina_Water,Muffin_Pastry_Case,Extra_Syrup,Bottled_Soda_Mt_Dew,...,Starbucks_DS_Mocha,Hot_Chocolate_16_oz,Dr_Pepper,Ocean_Spray_CranGrape,Gum_Peppermint,meal,year,month,day,hour
0,MNC7,2019-05-02 11:08:00,4.06,0.0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,2019,5,3,11
1,X1WE,2019-05-02 11:09:00,1.74,0.0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,2019,5,3,11
2,8V7V,2019-05-02 11:11:00,0.25,0.0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,2019,5,3,11
3,81U1,2019-05-02 11:17:00,2.20,0.0,0,1,0,0,0,0,...,0,0,0,0,0,NaN,2019,5,3,11
4,DTQ8,2019-05-02 11:23:00,8.01,0.0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,2019,5,3,11


In [24]:
# Mock Test using Training Set
Y = train['meal']
X = train.drop(['meal', 'id', 'DateTime'], axis=1)

x, xt, y, yt = train_test_split(X, Y, test_size=0.1, random_state = 42)

# Simple Decision Tree
tree = DecisionTreeClassifier(max_depth =5, min_samples_leaf=10)
tclf = tree.fit(x, y)
tpred = tclf.predict(xt)
print("\nThe decision tree has an accuracy of : %s\n"% str(accuracy_score(tpred, yt)))

# Bagging Ensemble
bag = BaggingClassifier(n_estimators=100, n_jobs=-1, random_state=42) 
baclf = bag.fit(x, y)
bapred = baclf.predict(xt)
print("\nThe bagging algorithm has an accuracy of : %s\n"% str(accuracy_score(bapred, yt)))

# Random Forest
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
fclf = forest.fit(x,y)
fpred = fclf.predict(xt)
print("\nThe random forest has an accuracy of : %s\n"% str(accuracy_score(fpred, yt)))

# Extreme Boosting; Boosted Trees
xgb = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.5, objective='binary:logistic')
xclf = xgb.fit(x,y)
xpred = xclf.predict(xt)
print("\nThe boosted tree has an accuracy of : %s\n"% str(accuracy_score(xpred, yt)))



The decision tree has an accuracy of : 0.8609738884968243


The bagging algorithm has an accuracy of : 0.8856739590684545


The random forest has an accuracy of : 0.8807339449541285


The boosted tree has an accuracy of : 0.881439661256175



In [27]:
# Defining Factors
trY = train['meal']
trX = train.drop(['meal', 'id', 'DateTime'], axis=1)
teY = test['meal']
teX = test.drop(['meal', 'id', 'DateTime'], axis=1)

# Simple Decision Tree
x, xt, y, yt = train_test_split(X, Y, test_size=0.1, random_state = 42)
tree = DecisionTreeClassifier(max_depth = 5, min_samples_leaf=10)
tclf = tree.fit(trX, trY)
tpred = tclf.predict(teX)

# Bagging Ensemble
bag = BaggingClassifier(n_estimators=100, n_jobs=-1, random_state=42) 
baclf = bag.fit(trX, trY)
bapred = baclf.predict(teX)

# Random Forest
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
fclf = forest.fit(trX, trY)
fpred = fclf.predict(teX)

# Extreme Boosting; Boosted Trees
xgb = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.5, objective='binary:logistic')
xclf = xgb.fit(trX, trY)
xpred = xclf.predict(teX)

print(len(xpred))


1000
